In [ ]:
import subprocess
import atexit
import threading
import requests
import sseclient
import json

url = "http://127.0.0.1:5175/v1/chat/completions"

process = subprocess.Popen([
    "./llama/llama-server",
    "-m", "./llama/models/Qwen3-14B-Q5_K_M.gguf",
    "-b", "512",
    "--n-gpu-layers", "41",
    "-t", "6",
    "-n", "-1",
    "--port", "5175",
    "--verbose"
], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, bufsize=1, text=True, encoding="utf-8")

atexit.register(process.terminate)

<bound method Popen.terminate of <Popen: returncode: None args: ['./llama/llama-server', '-m', './llama/model...>>

In [2]:
def log_output(pipe):
    for line in pipe:
        print(line, end="", flush=True)

thread = threading.Thread(target=log_output, args=(process.stdout,), daemon=True)
thread.start()

./llama/llama-server: /lib64/libcurl.so.4: no version information available (required by ./llama/llama-server)
load_backend: loaded RPC backend from /home/qbsoon/repos/LMnGen/llama/libggml-rpc.so
ggml_vulkan: Found 1 Vulkan devices:
ggml_vulkan: 0 = NVIDIA GeForce RTX 4070 Ti SUPER (NVIDIA) | uma: 0 | fp16: 1 | bf16: 0 | warp size: 32 | shared memory: 49152 | int dot: 1 | matrix cores: NV_coopmat2
load_backend: loaded Vulkan backend from /home/qbsoon/repos/LMnGen/llama/libggml-vulkan.so
load_backend: loaded CPU backend from /home/qbsoon/repos/LMnGen/llama/libggml-cpu-icelake.so


In [5]:
process.stdin.write("I'm glad you're working properly'\n")

34

In [2]:
history = [{"role": "system", "content": "You are a helpful assistant."}]

def send_prompt(prompt):
    history.append({"role": "user", "content": prompt})
    request = {"messages": history, "max_tokens": -1, "stream": True}
    response = requests.post(url, json=request, stream=True)
    client = sseclient.SSEClient(response)
    reply = ""
    final_stats = 0
    for event in client.events():
        if event.data.strip() == "[DONE]":
            break
        try:
            chunk = json.loads(event.data)
            choices = chunk.get('choices', [])
            if not choices:
                if "usage" in chunk or "timings" in chunk:
                    final_stats = chunk
                    yield chunk
                continue
            delta = choices[0].get('delta', {})
            token = delta.get('content')
            if token:
                reply += token
                yield token
        except (json.JSONDecodeError, KeyError):
            continue
    history.append({"role": "assistant", "content": reply.split("</think>")[1]})

In [4]:
reply_tokens = []
responsese = send_prompt("What are the best 5 frameworks for language management in JS?")
print(responsese)

<generator object send_prompt at 0x7f058e5b8170>


In [3]:
reply_tokens = []
responsese = send_prompt("What are the best 5 frameworks for language management in JS?")
stats = 0
for token in responsese:
    if isinstance(token, dict):
        stats = token
        continue
    print(token, end='', flush=True)
    reply_tokens.append(token)

print("\nSpeed: ", round(stats['timings']['predicted_per_second'], 2), "t/s")

<think>
Okay, the user is asking for the best 5 frameworks for language management in JavaScript. Let me think about this.

First, I need to clarify what they mean by "language management." That could be internationalization (i18n) and localization (l10n), right? So frameworks that help with translating text, formatting dates, numbers, etc., for different languages.

Now, I should recall the popular JS libraries for i18n. The first one that comes to mind is i18next. It's widely used and has good support for React and other frameworks. Then there's FormatJS, which is part of the React ecosystem and includes tools like react-intl. 

Another one is Transifex, but wait, Transifex is more of a translation management system rather than a framework. Maybe the user is looking for actual JS libraries, not CMS or platforms. So I should focus on libraries.

Then there's Preact's use of internationalization, but maybe that's too specific. Oh, and there's also Locale, but I think that's more of a u

In [ ]:
from PyQt6.QtWidgets import (
	QApplication, QWidget, QVBoxLayout, QHBoxLayout, QPushButton, QLabel, QTextEdit, QLineEdit,
	QTabWidget, QMessageBox, QTableWidget, QTableWidgetItem, QSizePolicy
)
import sys

class App(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("LMnGen App")
        self.chatHistory = [{"role": "system", "content": "You are a helpful assistant."}]

        self.mainLayout = QVBoxLayout()
        self.initUI()
        self.setLayout(self.mainLayout)

    def initUI(self):
        widget = QWidget()
        layout = QVBoxLayout()
        layout.setSpacing(6)

        self.chatDisplay = QTextEdit()
        self.chatDisplay.setReadOnly(True)
        self.chatDisplay.setStyleSheet("""
QTextEdit {
    background-color: #f0f0f0;
}
QScrollBar:vertical {
    background: #f0f0f0;
    width: 12px;
    margin: 0;
    border-radius: 6px;
}
QScrollBar::handle:vertical {
    background: #6c8cd5;
    min-height: 30px;
    border-radius: 6px;
}
QScrollBar::handle:vertical:hover {
    background: #3a5bbb;
}
QScrollBar::sub-line:vertical, QScrollBar::add-line:vertical {
    height: 0px;
}
QScrollBar::add-page:vertical, QScrollBar::sub-page:vertical {
    background: none;
}
""")
        layout.addWidget(self.chatDisplay)

        inputLayout = QHBoxLayout()
        self.chatInput = QLineEdit()
        self.chatInput.returnPressed.connect(self.send_prompt)
        self.chatInput.setPlaceholderText("Type your prompt here...")

        sendBtn = QPushButton("Send")
        sendBtn.clicked.connect(self.send_prompt)

        inputLayout.addWidget(self.chatInput)
        inputLayout.addWidget(sendBtn)

        layout.addLayout(inputLayout)
        widget.setLayout(layout)
        self.mainLayout.addWidget(widget)

    def send_prompt(self):
        prompt = self.chatInput.text().strip()
        self.chatHistory.append({"role": "user", "content": prompt})
        request = {"messages": self.chatHistory, "max_tokens": -1, "stream": True}
        response = requests.post(url, json=request, stream=True)
        client = sseclient.SSEClient(response)
        reply = ""
        for event in client.events():
            if event.data.strip() == "[DONE]":
                break
            try:
                chunk = json.loads(event.data)
                choices = chunk.get('choices', [])
                if not choices:
                    if "usage" in chunk or "timings" in chunk:
                        yield chunk
                    continue
                delta = choices[0].get('delta', {})
                token = delta.get('content')
                if token:
                    reply += token
                    yield token
            except (json.JSONDecodeError, KeyError):
                continue
        self.chatHistory.append({"role": "assistant", "content": reply.split("</think>")[1]})
        self.chatInput.clear()
        self.update_chat_display()

    def update_chat_display(self):
        self.chatDisplay.clear()
        for message in self.chatHistory:
            role = message['role']
            content = message['content']
            if role == 'user':
                self.chatDisplay.append(f"<b>User:</b> {content}")
            elif role == 'assistant':
                self.chatDisplay.append(f"<b>Assistant:</b> {content}")
            elif role == 'system':
                self.chatDisplay.append(f"<b><i>System:</i></b> {content}")
        self.chatDisplay.verticalScrollBar().setValue(self.chatDisplay.verticalScrollBar().maximum())

if __name__ == "__main__":
	app = QApplication(sys.argv)
	window = App()
	window.resize(1000, 600)
	window.show()
	sys.exit(app.exec())